<a href="https://colab.research.google.com/github/taureliloome/tron/blob/master/notebooks/MSP/ks10163_MD1_TextClassificationWithBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/LUMII-AILab/NLP_Course/blob/main/notebooks/MSP/TextClassificationWithBERT.ipynb" target="_new"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Teksta klasificēšana ar loģistisko regresiju un BERT

Piezīme: Colab izpildlaika vides parametros izvēlieties bezmaksas GPU (T4).

In [2]:
!pip install transformers
!pip install datasets
!pip install scikit-learn

## GoEmotions datu kopa

* Publikācija: https://aclanthology.org/2020.acl-main.372/
* Oriģinālā datu kopa: https://github.com/google-research/google-research/tree/master/goemotions
* Priekšapstrādāta **EN** versija: https://huggingface.co/datasets/google-research-datasets/go_emotions
* Priekšapstrādāta **LV** versija: https://huggingface.co/datasets/AiLab-IMCS-UL/go_emotions-lv

## BERT modeļa izvēle un tekstvienību dalītāja ielāde

* Oficiālie Google BERT modeļi - `base` un `large` versijas: https://huggingface.co/google-bert
* Neoficiālas mazākas BERT versijas, piem., `small`: https://huggingface.co/prajjwal1/bert-small
* Latviešu valodai priekšapmācīts BERT modelis: https://huggingface.co/AiLab-IMCS-UL/lvbert
* u.c.

Piezīme: Obligāti jāizmanto modelim atbilstošais tekstvienību dalītājs (*tokenizer*).

In [3]:
from transformers import BertTokenizer

In [4]:
# Ielādē CPU atmiņā izvēlētā BERT modeļa tekstvienību dalītāju
#bert_tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

In [5]:
# Ielādē CPU atmiņā 2. uzdevuma prasīto AiLAb-IMCS-UL/lvbert BERT modeļa tekstvienību dalītāju
bert_tokenizer = BertTokenizer.from_pretrained('AiLab-IMCS-UL/lvbert')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/390 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Datu kopas ielāde un priekšapstrāde

In [6]:
from datasets import load_dataset

In [7]:
# def is_single_label(sample):
#     value = sample["labels"]
#     if isinstance(value, (list, tuple)):
#         return len(value) == 1
#     else:
#         return False

# def to_int_label(sample):
#     return {"labels": sample["labels"][0]}

# def tokenize(batch):
#     return bert_tokenizer(batch["text"], truncation=True)

In [8]:
### 5. uzd pielāgojums
def is_single_label(sample):
    value = sample["labels_ekman"]
    if isinstance(value, (list, tuple)):
        return len(value) == 1
    else:
        return False

def to_int_label(sample):
    return {"labels": sample["labels_ekman"][0]}

def tokenize(batch):
    return bert_tokenizer(batch["text"], truncation=True)

In [10]:
#data_set = load_dataset("google-research-datasets/go_emotions", "simplified")
data_set = load_dataset("AiLab-IMCS-UL/go_emotions-lv", "simplified_ekman")
# filtered_data_set = data_set.filter(is_single_label)
# flattened_data_set = filtered_data_set.map(to_int_label)

filtered_data_set = data_set.filter(is_single_label) ### 5. uzd pielāgojums
flattened_data_set = filtered_data_set.map(to_int_label) ### 5. uzd pielāgojums

tokenized_data_set = flattened_data_set.map(tokenize, batched=True)
final_data_set = tokenized_data_set.select_columns(["input_ids", "labels"])

print("data_set:", data_set["train"][0])
print("filtered_data_set:", filtered_data_set["train"][0])
print("flattened_data_set:", flattened_data_set["train"][0])
print("tokenized_data_set:", tokenized_data_set["train"][0])
print("final_data_set:", final_data_set["train"][0])

train_set = final_data_set["train"]
validation_set = final_data_set["validation"]
test_set = final_data_set["test"]

Filter:   0%|          | 0/43410 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5426 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5427 [00:00<?, ? examples/s]

Map:   0%|          | 0/39555 [00:00<?, ? examples/s]

Map:   0%|          | 0/4946 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/39555 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/4946 [00:00<?, ? examples/s]

Map:   0%|          | 0/4968 [00:00<?, ? examples/s]

data_set: {'lv_text': 'Mans mīļākais ēdiens ir viss, kas man nebija jāgatavo pašai.', 'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [27], 'labels_ekman': [6], 'id': 'eebbqej'}
filtered_data_set: {'lv_text': 'Mans mīļākais ēdiens ir viss, kas man nebija jāgatavo pašai.', 'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [27], 'labels_ekman': [6], 'id': 'eebbqej'}
flattened_data_set: {'lv_text': 'Mans mīļākais ēdiens ir viss, kas man nebija jāgatavo pašai.', 'text': "My favourite food is anything I didn't have to cook myself.", 'labels': 6, 'labels_ekman': [6], 'id': 'eebbqej'}
tokenized_data_set: {'lv_text': 'Mans mīļākais ēdiens ir viss, kas man nebija jāgatavo pašai.', 'text': "My favourite food is anything I didn't have to cook myself.", 'labels': 6, 'labels_ekman': [6], 'id': 'eebbqej', 'input_ids': [2, 14546, 3254, 1332, 5840, 363, 21050, 695, 13810, 3021, 316, 238, 18795, 225, 138, 419, 2645, 26, 4191, 3094, 11160, 

## Loģistiskā regresija

### Izpildes vides sagatavošana

In [11]:
from transformers import BertModel

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import numpy as np
import torch

In [12]:
# Ielādē izvēlēto BERT modeli GPU atmiņā, inferences režīmā
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Izmantotais procesors:", device)

#bert_model = BertModel.from_pretrained("google-bert/bert-base-uncased").to(device).eval()
bert_model = BertModel.from_pretrained("AiLab-IMCS-UL/lvbert").to(device).eval()

Izmantotais procesors: cuda


config.json:   0%|          | 0.00/774 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

### Ievaddatu dalīšana tekstvienībās, kontekstuāla vektorizēšana

In [13]:
def get_embeddings(texts, batch_size=256, max_length=512):
    embeddings = []

    for i in range(0, len(texts), batch_size):
        tokenized_batch = bert_tokenizer(
            texts[i:i+batch_size],
            padding = True,
            truncation = True,
            max_length = max_length,
            return_tensors = "pt"
        )

        # Ielādē ieejas vērtības GPU/CPU
        tokenized_batch = {k: v.to(device) for k, v in tokenized_batch.items()}

        with torch.no_grad():
            out = bert_model(**tokenized_batch)
            cls = out.last_hidden_state[:, 0, :]  # [CLS] vektors

        embeddings.append(cls.detach().cpu().numpy())

    return np.vstack(embeddings)

In [14]:
X_train = get_embeddings(flattened_data_set["train"]["text"])
y_train = np.array(flattened_data_set["train"]["labels"])

X_valid = get_embeddings(flattened_data_set["validation"]["text"])
y_valid = np.array(flattened_data_set["validation"]["labels"])

X_test = get_embeddings(flattened_data_set["test"]["text"])
y_test = np.array(flattened_data_set["test"]["labels"])

In [15]:
# Atbrīvo GPU atmiņu (!)
del bert_model
torch.cuda.empty_cache()

### Klasifikatora apmācība un novērtēšana

In [16]:
clf = LogisticRegression(max_iter=10000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [17]:
print(accuracy_score(y_valid, clf.predict(X_valid)), "- validācijas kopa")
print(accuracy_score(y_test, clf.predict(X_test)), "- testa kopa")

0.5279013344116458 - validācijas kopa
0.5255636070853462 - testa kopa


Piezīme: Iegūtie tekstu jēdzienvektori, izmantojot priekšapmācītu BERT modeli bez pielāgošanas konkrētajam klasifikācijas uzdevumam, ir kontekstualizēti, taču modeļa svari ir iesaldēti, un tie loģistiskās regresijas apmācības laikā netiek uzdevumam pielāgoti. Tādēļ BERT bāzes modelis, visticamāk, neatspoguļo uzdevumam specifiskās nianses tik efektīvi, kā tad, ja to pielāgotu šim konkrētajam klasifikācijas uzdevumam.

## BERT pielāgošana

### Izpildes vides sagatavošana

In [18]:
from transformers import BertForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

### Sagatavošanās bāzes modeļa pielāgošanai

In [21]:
# Nosaka dažādo klašu skaitu apmācības datu  kopā
# label_count = len(data_set["train"].features["labels"].feature.names)

label_count = len(data_set["train"].features["labels_ekman"].feature.names) ### 5. uzd pielāgojums
print("label_count", label_count)

# Ielādē RAM izvēlēto BERT modeli, izveido tam atbilstošu klasificēšanas "galvu"
#bert_model = BertForSequenceClassification.from_pretrained(
#    'google-bert/bert-base-uncased', num_labels=label_count
#)

# Ielādē RAM izvēlēto BERT modeli, izveido tam atbilstošu klasificēšanas "galvu"
bert_model = BertForSequenceClassification.from_pretrained(
    'AiLab-IMCS-UL/lvbert', num_labels=label_count
)

# Nodefinē vienkāršotu novērtēšanas metriku - "accuracy"
def eval_metrics(p):
    preds = p.predictions.argmax(-1)
    return {"accuracy": float((preds == p.label_ids).mean())}

# Specificē modeļa apmācības hiperparametrus
args = TrainingArguments(
    #output_dir = "bert-base-uncased-go_emotions",
    output_dir = "lvbert-go_emotions-lv",
    learning_rate = 2e-5,              # tipiski BERT modeļiem
    per_device_train_batch_size = 64,  # atkarībā no GPU atmiņas; var ietekmēt rezultātu
    per_device_eval_batch_size = 128,  # atkarībā no GPU atmiņas
    num_train_epochs = 5,
    fp16 = True,                       # ātrdarbībai uz T4
    metric_for_best_model = "accuracy",
    save_strategy = "epoch",
    eval_strategy = "epoch",
    load_best_model_at_end = True,
    report_to = "none"                 # neizmantot W&B servisu
)

# Izveido apmācības "dzinēju"
trainer = Trainer(
    model = bert_model,
    args = args,
    train_dataset = train_set,
    eval_dataset = validation_set,
    compute_metrics = eval_metrics,
    processing_class = bert_tokenizer,
    data_collator = DataCollatorWithPadding(bert_tokenizer)
)

label_count 7


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at AiLab-IMCS-UL/lvbert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Bāzes modeļa pielāgošana klasificēšanas uzdevumam

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.091300,0.905123,0.672058
2,0.858800,0.873157,0.684998
3,0.742400,0.889164,0.681359
4,0.635400,0.942405,0.673474
5,0.456400,0.984562,0.676911


TrainOutput(global_step=3095, training_loss=0.7142058611301305, metrics={'train_runtime': 838.9582, 'train_samples_per_second': 235.739, 'train_steps_per_second': 3.689, 'total_flos': 5640488331300750.0, 'train_loss': 0.7142058611301305, 'epoch': 5.0})

### Labākās pielāgotās versijas testēšana

In [23]:
trainer.evaluate(test_set)

print("Labākais kontrolpunkts:", trainer.state.best_model_checkpoint)
print("Augstākā precizitāte:", trainer.state.best_metric)

Labākais kontrolpunkts: lvbert-go_emotions-lv/checkpoint-1238
Augstākā precizitāte: 0.6849979781641731


In [24]:
# Atbrīvo GPU atmiņu (!)
del bert_model
torch.cuda.empty_cache()